In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install -q langchain
!pip install -q tiktoken
!pip install -q transformers
!pip install -q huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00


In [8]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00


In [9]:
import os
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.docstore.document import Document
import tiktoken
from transformers import AutoTokenizer, pipeline
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline

In [12]:
hf_token_path="/content/drive/MyDrive/key/hf_token.txt"

os.environ["HF_TOKEN"] = open(hf_token_path, 'r', encoding='utf-8').read()
login(os.environ["HF_TOKEN"], add_to_git_credential=True)

#login(token=hf_token)
#model_name = "beomi/gemma-ko-2b"

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
text_path = "/content/drive/MyDrive/Colab Notebooks/rectified_text_2023_2.txt"
loader = TextLoader(text_path)
documents = loader.load()

In [14]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

문서개수: 1
첫번째 문서의 내용: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하였다. 일부 업권의 건전성 우려, 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다. 자산운용사 수신은 주가 상승, 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었다. 특히 비은행 예금 취급 기관 기업대출은 부동산 경기 부진 등으로 건설·부동산업 관련 대출 수요가 크게 둔화되면서 증가 규모가 전년 대비 큰 폭으로 축소되었다. 한편, 은행 기업대출의 경우 대기업 대출은 기업들이 회사채 대신 대출을 통한 자금 조달을 늘리면서 전년에 이어 높은 증가세를 보였으나, 중소기업 대출은 개인사업자에 대한 코로나19 지원 대출의 상환이 늘면서 증가 폭이 축소되었다. 가계대출은 주택 매매 거래 증가 및 대출 규제 완화, 주택금융공사의 특례 보금자리론 등 정책 금융상품 공급 확대에 힘입어 예금은행 주택담보대출과 기타 금융기관 대출이 늘어나면서 증가 전환하였다. 다만, 높은 금리 수준 및 총부채 원리금 상환 비율(DSR, Debt Service Ratio) 규제, 지방 부동산 경기 회복 지연 등의 영향으로 예금은행 기타 대출과 비은행 예금 취급 기관은 감소세를 지속하였다.경제 동향 금융·외환시장 외환시장 원/달러 환율 상승, 변동성 소폭 축소 2023년 말 원/달러 환율(1,288.

### Text Splitting

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter

# RecursiveCharacterTextSplitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 각 청크의 최대 문자 수
    chunk_overlap=50,  # 청크 간 중복되는 문자 수
    separators=["\n\n", "\n", ".", " "]  # 텍스트를 분할할 우선 순위 있는 구분자
)

# 텍스트 분할
splitted_documents = text_splitter.split_documents(documents)

In [16]:
# 분할된 텍스트 출력
for i, chunk in enumerate(splitted_documents):
    print(f"청크 {i + 1}:\n{chunk}\n")

청크 1:
page_content='기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하였다' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/rectified_text_2023_2.txt'}

청크 2:
page_content='. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하였다. 일부 업권의 건전성 우려, 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다. 자산운용사 수신은 주가 상승, 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었다' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/rectified_text_2023_2.txt'}

청크 3:
page_content='. 특히 비은행 예금 취급 기관 기업대출은 부동산 경기 부진 등으로 건설·부동산업 관련 대출 수요가 크게 둔화되면서 증가 규모가 전년 대비 큰 폭으로 축소되었다. 한편, 은행 기업대출의 경우 대기업 대출은 기업들이 회사채 대신 대출을 통한 자금 조달을 늘리면서 전년에 이어 높은 증가세를 보였으나, 중소기업 대출은 개인사업자에 대한 코로나19 지원 대출의 상환이 늘면서 증가 폭이 축소되었다' metadata={'source': '/cont

In [3]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00


In [17]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
from langchain import FAISS
import faiss
import numpy as np

In [4]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.3 MB/s eta 0:00:00


In [18]:
from sentence_transformers import SentenceTransformer

# SentenceTransformer 모델 로드
embedding_model_test = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = [doc.page_content for doc in splitted_documents]
embeddings_test = embedding_model_test.embed_documents(texts)
d = len(embeddings_test[0]) #벡터 차원 지정
index = faiss.IndexFlatL2(d)  # 백터 간 유사성 계산할 수 있는 FAISS 인덱스 생성 (L2 기반)
embeddings_test_array = np.array(embeddings_test).astype('float32')
index.add(embeddings_test_array)
#index.add(np.array(embeddings)) # 변환된 벡터 임베딩을 FAISS 인덱스에 추가

<ipython-input-18-9600a6bb4e29>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model_test = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
docstore = InMemoryDocstore({i: Document(page_content=texts[i]) for i in range(len(texts))})
faiss_index = FAISS(embedding_function=embedding_model_test, index=index, docstore=docstore, index_to_docstore_id={i: i for i in range(len(texts))})

In [20]:
query = "기준 금리에 대해 설명해주세요"
results = faiss_index.similarity_search(query, k=5)
for result in results:
    print(f'#############################')
    print(f'문서 내용: {result.page_content}')

#############################
문서 내용: . 대외 측면에서는 미연준이 금리 인상을 사실상 종료하면서 외환 부문 리스크가 완화되었지만 이스라엘-하마스 전쟁이 장기화되고 홍해 지역 긴장이 높아지는 등 지정학적 리스크와 관련한 불확실성은 지속되었다
#############################
문서 내용: . 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다루었다.
#############################
문서 내용: . 전략 지원 부문은 지역 경제 여건 및 특성을 감안하여 중점 지원이 필요하다고 각 지역 본부장이 인정한 지역 전략 산업 및 지역 특화 산업 등에 대하여 금융 기관 대출 취급 실적의 50%를 지원하고, 특별 지원 부문은 각 지역 본부장이 선정한 경기 부진 업종의 운전자금 대출에 한하여 금융 기관 대출 취급 실적의 50%를 지원한다. 일반 지원 부문은 전략 및 특별 지원 부문 이외에 기타 지원이 필요하다고 각 지역 본부장이 인정한 부문에 대하여 금융 기관 대출 취급 실적의 50% 이내에서 비례 배분으로 지원한다
#############################
문서 내용: . 다만, 창업 기업 지원 부문과 관련하여 ‘기술 평가 품질 관리 위원회’ 심사 결과 반영은 2024년 상반기 심사 결과가 나온 날부터 시행하기로 하였다. 통합 고용 세액 공제는 지원 대상 기업 요건에 포함되고, 지원 대상을 직전 과세 연도에 ‘세액 공제를 받은 기업’ 대신 ‘공제 세액이 발생한 기업’으로 변경하였다. 2023년 말 기준 금융 중개 지원 대출은 무역 금융 지원 프로그램, 신성장·일자리 지원 프로그램, 중소기업 대출 안정화 프로그램, 지방 중소기업 지원 프로그램 등으로 총 한도는 30.0조 원 규모로 운용하고 있다
#############################
문서 내용: 한국은행은 콜금리(익일물)를 금융통화위원회가 정한 기준금리 수준에서 유지하기 위해 통화안정증권, 환매조건부증권(RP), 통화안정계정 등 다

In [5]:
!pip install -U pyarrow

In [6]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00


In [21]:
from sentence_transformers.readers import InputExample
from datasets import load_dataset

# KLUE-STS 데이터셋 로드
klue_sts_train = load_dataset("klue", "sts", split='train[:90%]')
klue_sts_valid = load_dataset("klue", "sts", split='train[-10%:]')

# InputExample 변환
def make_sts_input_example(dataset):
    input_examples = []
    for i, data in enumerate(dataset):
        sentence1 = data['sentence1']
        sentence2 = data['sentence2']
        score = (data['labels']['label']) / 5.0  # 0~5 범위를 0~1로 정규화
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))
    return input_examples

sts_train_examples = make_sts_input_example(klue_sts_train)
sts_valid_examples = make_sts_input_example(klue_sts_valid)

README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/68.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

In [24]:
from sentence_transformers import SentenceTransformer, models, losses
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

sts_output_path = "/content/drive/MyDrive/models/sts-finetuned-model"

# 사전 학습된 모델과 Pooling 레이어 로드
embedding_model = models.Transformer(
    model_name_or_path="klue/roberta-base",
    max_seq_length=256,
    do_lower_case=True
)

# Mean Pooling 사용
pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)

model = SentenceTransformer(modules=[embedding_model, pooling_model])

# DataLoader 생성
train_dataloader = DataLoader(sts_train_examples, shuffle=True, batch_size=32)

# CosineSimilarityLoss 사용
train_loss = losses.CosineSimilarityLoss(model=model)

# 학습 시작
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=6,
    evaluator=EmbeddingSimilarityEvaluator.from_input_examples(sts_valid_examples, name="sts-dev"),
    evaluation_steps=1000,
    output_path=sts_output_path
)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine,Sts-dev Pearson Manhattan,Sts-dev Spearman Manhattan,Sts-dev Pearson Euclidean,Sts-dev Spearman Euclidean,Sts-dev Pearson Dot,Sts-dev Spearman Dot,Sts-dev Pearson Max,Sts-dev Spearman Max
329,No log,No log,0.908893,0.889168,0.900387,0.884351,0.900088,0.884224,0.880019,0.860891,0.908893,0.889168
658,0.118500,No log,0.931353,0.893240,0.922281,0.892550,0.922155,0.892412,0.920976,0.878917,0.931353,0.893240
987,0.118500,No log,0.946782,0.907944,0.940038,0.907351,0.939988,0.907412,0.940281,0.896267,0.946782,0.907944
1000,0.020100,No log,0.947069,0.908194,0.940200,0.907484,0.940150,0.907544,0.940542,0.896739,0.947069,0.908194
1316,0.020100,No log,0.955106,0.916342,0.949314,0.915325,0.949385,0.915689,0.949697,0.906127,0.955106,0.916342
1645,0.012300,No log,0.958296,0.918635,0.952748,0.917717,0.952864,0.918032,0.953196,0.908344,0.958296,0.918635
1974,0.012300,No log,0.960452,0.921604,0.954449,0.920499,0.954608,0.920862,0.955487,0.911010,0.960452,0.921604


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [25]:
import random


nli_output_path = "/content/drive/MyDrive/models/nli-finetuned-model"
nli_sts_output_path = "/content/drive/MyDrive/models/nli-sts-finetuned-model"

# NLI 데이터 로드 및 처리
klue_nli_train = load_dataset("klue", "nli", split='train')


# Triplet 형태로 변환
def make_nli_triplet_input_example(dataset):
    train_data = {}
    def add_to_samples(sent1, sent2, label):
        if sent1 not in train_data:
            train_data[sent1] = {'contradiction': set(), 'entailment': set(), 'neutral': set()}
        train_data[sent1][label].add(sent2)

    for i, data in enumerate(dataset):
        sent1 = data['hypothesis'].strip()
        sent2 = data['premise'].strip()
        if data['label'] == 0:
            label = 'entailment'
        elif data['label'] == 1:
            label = 'neutral'
        else:
            label = 'contradiction'
        add_to_samples(sent1, sent2, label)
        add_to_samples(sent2, sent1, label)  # 반대 관계도 추가

    # InputExamples로 변환
    input_examples = []
    for sent1, others in train_data.items():
        if len(others['entailment']) > 0 and len(others['contradiction']) > 0:
            input_examples.append(InputExample(texts=[sent1, random.choice(list(others['entailment'])), random.choice(list(others['contradiction']))]))
    return input_examples

nli_train_examples = make_nli_triplet_input_example(klue_nli_train)

# MultipleNegativesRankingLoss 사용한 NLI 학습
train_dataloader = DataLoader(nli_train_examples, shuffle=True, batch_size=32)

train_loss = losses.MultipleNegativesRankingLoss(model)


model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=2,
    output_path=nli_output_path
)

Step,Training Loss
500,0.678700


In [26]:
train_dataloader = DataLoader(sts_train_examples, shuffle=True, batch_size=32)
validation_dataloader = DataLoader(sts_valid_examples, shuffle=False, batch_size=32)

# Validation 성능 평가를 위한 evaluator 설정
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(sts_valid_examples, name="sts-dev")

# NLI 모델에 STS 추가 학습

model = SentenceTransformer(nli_output_path)
model.fit(
    train_objectives=[(train_dataloader, losses.CosineSimilarityLoss(model))],
    evaluator=evaluator,
    epochs=7,
    evaluation_steps=1000,
    output_path = nli_sts_output_path
)

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine,Sts-dev Pearson Manhattan,Sts-dev Spearman Manhattan,Sts-dev Pearson Euclidean,Sts-dev Spearman Euclidean,Sts-dev Pearson Dot,Sts-dev Spearman Dot,Sts-dev Pearson Max,Sts-dev Spearman Max
329,No log,No log,0.960051,0.921835,0.953284,0.920835,0.953403,0.920895,0.953472,0.909347,0.960051,0.921835
658,0.009600,No log,0.960973,0.921847,0.954378,0.920815,0.954549,0.921095,0.955345,0.910199,0.960973,0.921847
987,0.009600,No log,0.961035,0.921512,0.954447,0.920575,0.954635,0.920762,0.955796,0.910107,0.961035,0.921512
1000,0.006400,No log,0.961271,0.921762,0.954769,0.920856,0.954945,0.921202,0.956084,0.910314,0.961271,0.921762
1316,0.006400,No log,0.961621,0.923098,0.955072,0.921784,0.955273,0.922235,0.956426,0.911179,0.961621,0.923098
1645,0.005500,No log,0.962004,0.923131,0.955569,0.922211,0.955687,0.922564,0.956932,0.911868,0.962004,0.923131
1974,0.005500,No log,0.962386,0.923540,0.956155,0.923052,0.956325,0.923382,0.956977,0.911775,0.962386,0.923540
2000,0.004500,No log,0.962162,0.922604,0.955637,0.921805,0.955828,0.922080,0.956509,0.910417,0.962162,0.922604
2303,0.004500,No log,0.962662,0.924788,0.955517,0.923408,0.955644,0.923581,0.957362,0.913016,0.962662,0.924788


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [27]:
# Fine-tuned 모델 로드
ft_model = SentenceTransformer(nli_sts_output_path)


In [28]:
texts = [doc.page_content for doc in splitted_documents]
embeddings = ft_model.encode(texts)
d = len(embeddings[0]) #벡터 차원 지정
index = faiss.IndexFlatL2(d)  # 백터 간 유사성 계산할 수 있는 FAISS 인덱스 생성 (L2 기반)
index.add(np.array(embeddings)) # 변환된 벡터 임베딩을 FAISS 인덱스에 추가

In [32]:
# 쿼리문 작성
query = "기준 금리에 대해 설명해주세요"

# 쿼리를 벡터화
query_embedding = ft_model.encode([query])


# 유사한 문서 검색
k=3
distances, indices = index.search(np.array(query_embedding), k)

# 검색된 문서 출력
print("Query:", query)
print("\nTop 3 most similar documents:\n")
for i in range(k):
    print(f"Rank {i+1}:")
    print(f"Document: {texts[indices[0][i]]}")
    print(f"Distance: {distances[0][i]}\n")

Query: 기준 금리에 대해 설명해주세요

Top 3 most similar documents:

Rank 1:
Document: 나. 기준금리 한국은행은 2023년 중 성장세를 점검하면서 중기적 시계에서 물가 상승률이 목표 수준(2%)에서 안정될 수 있도록 기준금리를 긴축적인 수준에서 운용하였다. 이 과정에서 인플레이션 둔화 흐름, 금융 안정 측면의 리스크와 성장의 하방 위험, 가계부채 추이, 그간의 금리 인상 파급 효과, 주요국의 통화정책 변화, 지정학적 리스크 등을 면밀히 점검하였다. 기간별로 살펴보면, 우선 1월에는 기준금리를 연 3.25%에서 3.50%로 0.25% 포인트 인상하였다
Distance: 127.73157501220703

Rank 2:
Document: 한국은행이 수행한 통화신용정책의 상세한 내용은 한국은행이 발간하는 ｢통화신용정책보고서(분기)｣의 ｢Ⅱ. 통화신용정책 운영｣에서 확인할 수 있다. 한국은행의 통화정책 체계에 대한 상세한 설명은 한국은행이 발간한 ｢한국의 통화정책(2017)｣에서 확인할 수 있다.업무 현황 통화신용정책이라고 판단하였기 때문이다. 2월 이후에는 물가상승률의 기조적인 둔화 흐름을 이어갔지만 목표 수준을 상회하는 오름세가 연중 이어짐에 따라 기준금리를 긴축적인 수준인 3.5%에서 유지하면서 추가 인상 필요성을 판단하였다
Distance: 131.4873046875

Rank 3:
Document: 기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융·외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각적으로 분석하